In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/ID_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3132717110,ID,"700 E FRANKLIN RD, MERIDIAN, ID",43.605187,-116.382836,1.600101e+14
1,4037657101,ID,"1111 JEFFERSON ST, BOISE, ID",43.619731,-116.205940,1.600100e+14
2,4034667109,ID,"3330 E. Louise Dr, Suite 300, MERIDIAN, ID",43.604024,-116.353172,1.600101e+14
3,3029957103,ID,"2095 E. Commercial Street, MERIDIAN, ID",43.609116,-116.368114,1.600101e+14
4,4086377107,ID,"250 S. Beechwood Ave., #201, BOISE, ID",43.602477,-116.272347,1.600100e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3132717110,ID,"700 E FRANKLIN RD, MERIDIAN, ID",43.605187,-116.382836,1.600101e+14
1,4037657101,ID,"1111 JEFFERSON ST, BOISE, ID",43.619731,-116.205940,1.600100e+14
2,4034667109,ID,"3330 E. Louise Dr, Suite 300, MERIDIAN, ID",43.604024,-116.353172,1.600101e+14
3,3029957103,ID,"2095 E. Commercial Street, MERIDIAN, ID",43.609116,-116.368114,1.600101e+14
4,4086377107,ID,"250 S. Beechwood Ave., #201, BOISE, ID",43.602477,-116.272347,1.600100e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
368,5223057107,ID,"1805 SW 4th Ave, FRUITLAND, ID",44.025265,-116.991226,NaN
1878,6559698606,ID,"20158 Highway 93, Carey, ID",46.678546,-114.074737,NaN
2295,6866047004,ID,"20158 hwy 93, CAREY, ID",46.678546,-114.074737,NaN
2478,1944747104,ID,"10942 W State St, STAR, ID",43.692224,-116.490132,NaN
2967,3378157108,ID,"208 Spruce Ave Unit B - 4c, KETCHUM, ID",43.680740,-114.363662,NaN
3413,9480637310,ID,"2502 W KUNA CAVE RD, KUNA, ID",43.427822,-116.447486,NaN
4347,4859258508,ID,"9169 W STATE ST, GARDEN CITY, ID",43.677031,-116.299249,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('16')] #ID
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/7 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

7


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_14152/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
368,5223057107,ID,"1805 SW 4th Ave, FRUITLAND, ID",44.025265,-116.991226,[]
1878,6559698606,ID,"20158 Highway 93, Carey, ID",46.678546,-114.074737,[]
2295,6866047004,ID,"20158 hwy 93, CAREY, ID",46.678546,-114.074737,[]
2478,1944747104,ID,"10942 W State St, STAR, ID",43.692224,-116.490132,[160010102271020]
2967,3378157108,ID,"208 Spruce Ave Unit B - 4c, KETCHUM, ID",43.680740,-114.363662,[160139603013068]
3413,9480637310,ID,"2502 W KUNA CAVE RD, KUNA, ID",43.427822,-116.447486,[160010105011143]
4347,4859258508,ID,"9169 W STATE ST, GARDEN CITY, ID",43.677031,-116.299249,[160010102363010]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_14152/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
368,5223057107,ID,"1805 SW 4th Ave, FRUITLAND, ID",44.025265,-116.991226,NaN
1878,6559698606,ID,"20158 Highway 93, Carey, ID",46.678546,-114.074737,NaN
2295,6866047004,ID,"20158 hwy 93, CAREY, ID",46.678546,-114.074737,NaN
2478,1944747104,ID,"10942 W State St, STAR, ID",43.692224,-116.490132,160010102271020
2967,3378157108,ID,"208 Spruce Ave Unit B - 4c, KETCHUM, ID",43.680740,-114.363662,160139603013068
3413,9480637310,ID,"2502 W KUNA CAVE RD, KUNA, ID",43.427822,-116.447486,160010105011143
4347,4859258508,ID,"9169 W STATE ST, GARDEN CITY, ID",43.677031,-116.299249,160010102363010


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
368,5223057107,ID,"1805 SW 4th Ave, FRUITLAND, ID",44.025265,-116.991226,NaN
1878,6559698606,ID,"20158 Highway 93, Carey, ID",46.678546,-114.074737,NaN
2295,6866047004,ID,"20158 hwy 93, CAREY, ID",46.678546,-114.074737,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
368,5223057107,NaN
1878,6559698606,NaN
2295,6866047004,NaN
2478,1944747104,160010102271020
2967,3378157108,160139603013068
3413,9480637310,160010105011143
4347,4859258508,160010102363010


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/ID_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,303724,3132717110,ID,"700 E FRANKLIN RD, MERIDIAN, ID",43.605187,-116.382836,1.600101e+14
1,303725,4037657101,ID,"1111 JEFFERSON ST, BOISE, ID",43.619731,-116.205940,1.600100e+14
2,303726,4034667109,ID,"3330 E. Louise Dr, Suite 300, MERIDIAN, ID",43.604024,-116.353172,1.600101e+14
3,303727,3029957103,ID,"2095 E. Commercial Street, MERIDIAN, ID",43.609116,-116.368114,1.600101e+14
4,303728,4086377107,ID,"250 S. Beechwood Ave., #201, BOISE, ID",43.602477,-116.272347,1.600100e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,303724,3132717110,ID,"700 E FRANKLIN RD, MERIDIAN, ID",43.605187,-116.382836,1.600101e+14,NaN
1,303725,4037657101,ID,"1111 JEFFERSON ST, BOISE, ID",43.619731,-116.205940,1.600100e+14,NaN
2,303726,4034667109,ID,"3330 E. Louise Dr, Suite 300, MERIDIAN, ID",43.604024,-116.353172,1.600101e+14,NaN
3,303727,3029957103,ID,"2095 E. Commercial Street, MERIDIAN, ID",43.609116,-116.368114,1.600101e+14,NaN
4,303728,4086377107,ID,"250 S. Beechwood Ave., #201, BOISE, ID",43.602477,-116.272347,1.600100e+14,NaN
...,...,...,...,...,...,...,...,...
4410,308134,8528738503,ID,"108 Airport Rd, Kamiah, ID",46.218843,-116.016474,1.604994e+14,NaN
4411,308135,8585757008,ID,"313 N Spokane Street, POST FALLS, ID",47.710015,-116.948526,1.605500e+14,NaN
4412,308136,9002167110,ID,"PO BOX 3869, HAILEY, ID",43.519629,-114.315325,1.601396e+14,NaN
4413,308137,9060577102,ID,"13403 W GOVERNMENT WAY, HAYDEN, ID",47.793812,-116.787280,1.605500e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,303724,3132717110,ID,"700 E FRANKLIN RD, MERIDIAN, ID",43.605187,-116.382836,1.600101e+14,NaN,160010103682062.0
1,303725,4037657101,ID,"1111 JEFFERSON ST, BOISE, ID",43.619731,-116.205940,1.600100e+14,NaN,160010001021053.0
2,303726,4034667109,ID,"3330 E. Louise Dr, Suite 300, MERIDIAN, ID",43.604024,-116.353172,1.600101e+14,NaN,160010103701013.0
3,303727,3029957103,ID,"2095 E. Commercial Street, MERIDIAN, ID",43.609116,-116.368114,1.600101e+14,NaN,160010103692012.0
4,303728,4086377107,ID,"250 S. Beechwood Ave., #201, BOISE, ID",43.602477,-116.272347,1.600100e+14,NaN,160010020003010.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3132717110,ID,"700 E FRANKLIN RD, MERIDIAN, ID",43.605187,-116.382836,160010103682062.0
1,4037657101,ID,"1111 JEFFERSON ST, BOISE, ID",43.619731,-116.205940,160010001021053.0
2,4034667109,ID,"3330 E. Louise Dr, Suite 300, MERIDIAN, ID",43.604024,-116.353172,160010103701013.0
3,3029957103,ID,"2095 E. Commercial Street, MERIDIAN, ID",43.609116,-116.368114,160010103692012.0
4,4086377107,ID,"250 S. Beechwood Ave., #201, BOISE, ID",43.602477,-116.272347,160010020003010.0
...,...,...,...,...,...,...
4410,8528738503,ID,"108 Airport Rd, Kamiah, ID",46.218843,-116.016474,160499400002003.0
4411,8585757008,ID,"313 N Spokane Street, POST FALLS, ID",47.710015,-116.948526,160550004024040.0
4412,9002167110,ID,"PO BOX 3869, HAILEY, ID",43.519629,-114.315325,160139601023008.0
4413,9060577102,ID,"13403 W GOVERNMENT WAY, HAYDEN, ID",47.793812,-116.787280,160550002022016.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
368,5223057107,ID,"1805 SW 4th Ave, FRUITLAND, ID",44.025265,-116.991226,NaN
1878,6559698606,ID,"20158 Highway 93, Carey, ID",46.678546,-114.074737,NaN
2295,6866047004,ID,"20158 hwy 93, CAREY, ID",46.678546,-114.074737,NaN


Convert to File

In [19]:
fips_ID = fips_merge_drop

In [20]:
fips_ID.to_csv("../../../data/state_data/geo/geo_fips/23/ID_fips_scraped.csv") 